In [84]:
import pandas as pd
import collections

import numpy as np
import folium as fo
import json
from bokeh.io import show,  output_notebook, export_png
from bokeh.plotting import figure
from bokeh.models import LabelSet,Legend
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [85]:
df=pd.read_parquet('resources/cleaned_franchises.parquet')
df['Results'] = (df['Results'] == 'Fail')
df = df.rename(columns={'Results': 'Fails'})
df['Inspection Date'] = df['Inspection Date'].apply(lambda x: x.year)
df = df[df['Inspection Date'] !=2018]
df = df[df['Inspection Date']!=2019]
df.head()

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Fails,...,Re-inspection,Community,Neighborhood,Ward,Precinct,Violation IDs,Nb Violations,Nb Violations of high severity,Nb Violations of medium severity,Nb Violations of low severity
Inspection ID,,,,,,,,,,,,,,,,,,,,,
2129816,TC HOTDOGS,TCHOTDOGS,1954116,Restaurant,Medium,1740 N KOSTNER AVE,60639,2017,Canvass,False,...,True,HUMBOLDT PARK,HUMBOLDT PARK,36,12,"[2, 32, 38]",3,1,0,2
2129807,Hash,HASH,2198209,Restaurant,High,1357 N WESTERN AVE,60622,2017,Canvass,False,...,True,WEST TOWN,"WICKER PARK,WEST TOWN",1,14,[35],1,0,0,1
2129810,PICCOLO MONDO CAFE,PICCOLOMONDOCAFE,1908007,Restaurant,High,1642 E 56TH ST,60637,2017,Suspect food poisoning,True,...,False,HYDE PARK,HYDE PARK,5,35,"[3, 18, 34, 35, 38]",5,1,1,3
2129812,GO! GROCER,GOGROCER,2163021,Grocery store,Low,2930 N SHERIDAN RD,60657,2017,Canvass,False,...,False,LAKE VIEW,LAKE VIEW,44,8,"[12, 32, 38]",3,1,0,2
2129817,COSI,COSI,1597223,Restaurant,High,33 N DEARBORN ST,60602,2017,Canvass,False,...,True,LOOP,LOOP,42,33,"[32, 34, 35]",3,0,0,3


In [86]:
nb_licenses_by_aka_dict = {}
for name in df['AKA Name'].unique():
    nb_uniques = len(df[df['AKA Name'] == name].groupby('License #'))
    nb_licenses_by_aka_dict[name] = nb_uniques
nb_licenses_by_aka = pd.DataFrame(nb_licenses_by_aka_dict.items(), columns=['AKA Name', 'Nb Licenses']).set_index('AKA Name').sort_values('Nb Licenses', ascending=False)
nb_licenses_by_aka.head(20)

,Nb Licenses
AKA Name,
SUBWAY,442
7ELEVEN,303
DUNKINDONUTS,283
STARBUCKS,233
MCDONALDS,183
BURGERKING,119
KFC,91
CITGO,85
HAROLDSCHICKEN,79


In [105]:
nb_licenses_by_aka.head(40)

,Nb Licenses
AKA Name,
SUBWAY,442
7ELEVEN,303
DUNKINDONUTS,283
STARBUCKS,233
MCDONALDS,183
BURGERKING,119
KFC,91
CITGO,85
HAROLDSCHICKEN,79


In [106]:
df_mc = df[df['AKA Name']== 'MCDONALDS']
df_bk = df[df['AKA Name']== 'BURGERKING']
df_wd = df[df['AKA Name']== 'WENDYS']
df_kfc = df[df['AKA Name']== 'KFC']
df_hc = df[df['AKA Name']== 'HAROLDSCHICKEN']
df_pp = df[df['AKA Name']== 'POPEYES']
df_sw = df[df['AKA Name']== 'SUBWAY']
df_ps = df[df['AKA Name']== 'POTBELLYSANDWICH']
df_ph = df[df['AKA Name']== 'PIZZAHUT']
df_dm = df[df['AKA Name']== 'DOMINOS']
df_pjp = df[df['AKA Name'] == 'PAPAJOHNSPIZZA']

In [107]:
col_to_drop=['DBA Name',
 'AKA Name',
 'Facility Type',
 'Violations',
 'License #',
 'Risk',
 'Address',
 'Zip',
 'Inspection Type',
 'Latitude',
 'Longitude',
 'Re-inspection',
 'Community',
 'Neighborhood',
 'Ward',
 'Precinct',
 'Violation IDs']
col_to_rename= {'Nb Violations of high severity': 'high', 'Nb Violations of medium severity': 'medium', 'Nb Violations of low severity': 'low'}

In [108]:
def nb_inspections (data,franchise):
    nb_inspections_by_year = data[['Inspection Date', 'License #']].copy()
    nb_inspections_by_year.rename(columns={'Inspection Date' : 'Inspection Year'}, inplace=True)
    nb_inspections_by_year = nb_inspections_by_year.groupby('Inspection Year').count().rename(columns={'License #' : 'Count'})
    nb_inspections_by_year.Count = nb_inspections_by_year.Count/nb_licenses_by_aka.loc[franchise,"Nb Licenses"]
    return nb_inspections_by_year

In [109]:
def df_violations(data):
    df_violations = data.drop(col_to_drop,axis=1).rename(columns =col_to_rename)
    df_violations ['Inspections'] = 1
    df_violations  = df_violations .set_index('Inspection Date')
    return df_violations


In [110]:
def agg_year(violations,inspections):
    agg_year = violations.groupby('Inspection Date').agg(sum)
    agg_year['high'] = agg_year.high / agg_year['Nb Violations']
    agg_year['medium'] = agg_year.medium / agg_year['Nb Violations']
    agg_year['low'] = agg_year.low / agg_year['Nb Violations']
    agg_year['inspection_fail_rate'] = agg_year.Fails / agg_year.Inspections
    agg_year['Mean inspections']=inspections.Count 
    return agg_year

In [112]:
nb_inspections_by_year_mc = nb_inspections(df_mc,'MCDONALDS')
nb_inspections_by_year_bk = nb_inspections(df_bk,'BURGERKING')
nb_inspections_by_year_wd = nb_inspections(df_wd,'WENDYS')
nb_inspections_by_year_kfc = nb_inspections(df_kfc,'KFC')
nb_inspections_by_year_hc = nb_inspections(df_hc,'HAROLDSCHICKEN')
nb_inspections_by_year_pp = nb_inspections(df_pp,'POPEYES')
nb_inspections_by_year_sw = nb_inspections(df_sw,'SUBWAY')
nb_inspections_by_year_ps = nb_inspections(df_ps,'POTBELLYSANDWICH')
nb_inspections_by_year_ph = nb_inspections(df_ph,'PIZZAHUT')
nb_inspections_by_year_dm = nb_inspections(df_dm,'DOMINOS')
nb_inspections_by_year_pjp = nb_inspections(df_pjp,'PAPAJOHNSPIZZA')

In [113]:
df_mc_violations = df_violations(df_mc)
df_bk_violations = df_violations(df_bk)
df_wd_violations = df_violations(df_wd)
df_kfc_violations = df_violations(df_kfc)
df_hc_violations = df_violations(df_hc)
df_pp_violations = df_violations(df_pp)
df_sw_violations = df_violations(df_sw)
df_ps_violations = df_violations(df_ps)
df_ph_violations = df_violations(df_ph)
df_dm_violations = df_violations(df_dm)
df_pjp_violations = df_violations(df_pjp)

In [114]:
agg_year_mc = agg_year(df_mc_violations,nb_inspections_by_year_mc)
agg_year_bk = agg_year(df_bk_violations,nb_inspections_by_year_bk)
agg_year_wd = agg_year(df_wd_violations,nb_inspections_by_year_wd)
agg_year_kfc = agg_year(df_kfc_violations,nb_inspections_by_year_kfc)
agg_year_pp = agg_year(df_pp_violations,nb_inspections_by_year_pp)
agg_year_hc = agg_year(df_hc_violations,nb_inspections_by_year_hc)
agg_year_sw = agg_year(df_sw_violations,nb_inspections_by_year_sw)
agg_year_ps = agg_year(df_ps_violations,nb_inspections_by_year_ps)
agg_year_ph = agg_year(df_ph_violations,nb_inspections_by_year_ph)
agg_year_dm = agg_year(df_dm_violations,nb_inspections_by_year_dm)
agg_year_pjp = agg_year(df_pjp_violations,nb_inspections_by_year_pjp)

In [115]:
colors = ['#FFC72C', '#DA291C','#8ED5E6', '#f5d4b7', '#183E07','#008C15','#000000']
years = list(nb_inspections_by_year_mc.index)
for i in range(len(years)): years[i]=str(years[i])
years
data = {'Years' : years,
        'MCDONALDS'   : nb_inspections_by_year_mc.Count,
        'BURGERKING'  : nb_inspections_by_year_bk.Count,
        'WENDYS'  : nb_inspections_by_year_wd.Count,
        'KFC' : nb_inspections_by_year_kfc.Count,
        'HAROLDSCHICKEN'  : nb_inspections_by_year_hc.Count,
       'SUBWAY': nb_inspections_by_year_sw.Count,
       'POTBELLYSANDWICH': nb_inspections_by_year_ps.Count}

In [123]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Inspection Fail Rate by year",
           toolbar_location=None)
r0 = p.line(x=years, y=agg_year_mc['inspection_fail_rate'],line_color=colors[0],line_width=3)
r1 = p.line(x=years, y=agg_year_bk['inspection_fail_rate'],line_color=colors[1],line_width=3)
rwd = p.line(x=years, y=agg_year_wd['inspection_fail_rate'],line_color=colors[2],line_width=3)
legend = Legend(items=[
    ('MCDONALDS',[r0]),
    ('BURGERKING',[r1]),
    ('WENDYS',[rwd]),
]
, location="center")
p.add_layout(legend, 'right')
export_png(p,filename='mc_bk_wd.png')
show(p)


In [124]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Inspection Fail Rate by year",
           toolbar_location=None)
r2 = p.line(x=years, y=agg_year_kfc['inspection_fail_rate'],line_color=colors[3],line_width=3)
r3 = p.line(x=years, y=agg_year_hc['inspection_fail_rate'],line_color=colors[4],line_width=3)
rpp = p.line(x=years, y=agg_year_pp['inspection_fail_rate'],line_color='#F68323',line_width=3)
legend = Legend(items=[
    ('KFC',[r2]),
    ('HAROLDSCHICKEN',[r3]),
    ('POPEYES',[rpp]),
]
, location="center")
p.add_layout(legend, 'right')
export_png(p,filename='kfc_hc_pp.png')
show(p)

In [125]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Inspection Fail Rate by year",
           toolbar_location=None)
r4 = p.line(x=years, y=agg_year_sw['inspection_fail_rate'],line_color=colors[5],line_width=3)
r5 = p.line(x=years, y=agg_year_ps['inspection_fail_rate'],line_color=colors[6],line_width=3)
legend = Legend(items=[
    ('SUBWAY',[r4]),
    ('POTBELLYSANDWICH',[r5]),
]
, location="center")
p.add_layout(legend, 'right')
show(p)

In [126]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Inspection Fail Rate by year",
           toolbar_location=None)
r4 = p.line(x=years, y=agg_year_ph['inspection_fail_rate'],line_color=colors[4],line_width=3)
r5 = p.line(x=years, y=agg_year_dm['inspection_fail_rate'],line_color=colors[5],line_width=3)
rpjp = p.line(x=years, y=agg_year_pjp['inspection_fail_rate'],line_color='#F68323',line_width=3)
legend = Legend(items=[
    ('PIZZAHUT',[r4]),
    ('DOMINOS',[r5]),
    ('PAPAJOHNSPIZZA',[rpjp]),
]
, location="center")
p.add_layout(legend, 'right')
export_png(p,filename='ph_dm_pjp.png')
show(p)

In [100]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Mean # of inspections by year",
           toolbar_location=None)
r6 = p.line(x=years, y=nb_inspections_by_year_mc.Count,line_color=colors[0],line_width=3)
r7 = p.line(x=years, y=nb_inspections_by_year_bk.Count,line_color=colors[1],line_width=3)
rwd = p.line(x=years, y=nb_inspections_by_year_wd.Count,line_color=colors[2],line_width=3)
r8 = p.line(x=years, y=nb_inspections_by_year_kfc.Count,line_color=colors[3],line_width=3)
r9 = p.line(x=years, y=nb_inspections_by_year_hc.Count,line_color=colors[4],line_width=3)
rpp = p.line(x=years, y=nb_inspections_by_year_pp.Count,line_color='#F68323',line_width=3)
r10 = p.line(x=years, y=nb_inspections_by_year_sw.Count,line_color=colors[5],line_width=3)
r11 = p.line(x=years, y=nb_inspections_by_year_ps.Count,line_color=colors[6],line_width=3)


legend = Legend(items=[
    ('MCDONALDS',[r6]),
    ('BURGERKING',[r7]),
    ('WENDYS',[rwd]),
    ('KFC',[r8]) ,
    ('HAROLDSCHICKEN',[r9]),
    ('POPEYES',[rpp]),
    ('SUBWAY',[r10]),
    ('POTBELLYSANDWICH',[r11]),]
, location="center")
p.add_layout(legend, 'right')
show(p)

The 'Sandwich' restaurant has more inspection by year that 'Burgers' and 'Chicken' restaurant. In addition, McDonalds has more inspections than Burger king but for the 'Chicken' restaurant, it's quite the same.